In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

set dictionary for data collecting

In [ ]:
head_coaches_of_the_year_dict = {
 'Season': [],
 'Person': [],
 'Team': [],
 'Age': [],
 'Nationality': []
}

collect data from the site

In [ ]:
head_coaches_of_the_year_url = 'https://basketball.realgm.com/nba/awards/by-type/Coach-of-the-Year/9001'
  
page = requests.get(head_coaches_of_the_year_url)
soup = BeautifulSoup(page.text, 'html.parser')
tables = soup.find_all('table')
rows = tables[2].findAll('tr')[1:]

for row in rows:
    cols = row.findAll('td')
    head_coaches_of_the_year_dict['Season'].append(cols[0].string)
    head_coaches_of_the_year_dict['Person'].append(cols[1].string)
    head_coaches_of_the_year_dict['Team'].append(cols[2].string)
    head_coaches_of_the_year_dict['Age'].append(cols[3].string)
    head_coaches_of_the_year_dict['Nationality'].append(cols[4].string)
    
df_head_coaches_of_the_year = pd.DataFrame(head_coaches_of_the_year_dict)

Edit & Add columns for future analysis, and save DataFrame to csv file

In [ ]:
df_head_coaches_of_the_year = df_head_coaches_of_the_year.replace('N/A','')
df_head_coaches_of_the_year['Season Short'] = df_head_coaches_of_the_year['Season'].str[:4]

In [ ]:
df_head_coaches_of_the_year.to_csv('NBA_coach_of_the_year.csv', index=False)